# Predicting Energy Prices Using Time Series Analysis

_By Isaac Valencia, Patrick de Beer, Cristina Ravenna, 
Mattias Escuerdo,and Matthew Henderson_
n


## Problem Statement and Motivation

## Intoduction and Description of Data

## Modeling Approach

## Project Trajectory, Results, and Interpretation

## Conclusion and Future Work